In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

sys.path.insert(0, '../src')
import classifier
import detector
from image import Image

In [3]:
from image import build_histogram_equalizer
TRAIN_DATA_DIR = os.path.abspath("../trainset")
COLORS = ['COLOR_STOP_SIGN_RED', 'COLOR_OTHER_RED',
          'COLOR_BROWN' , 'COLOR_ORANGE' ,
          'COLOR_BLUE' , 'COLOR_OTHER' ]

testset = set(map(str, [8, 11, 94, 95, 96, 97, 99, 100]))

data = {c: [] for c in COLORS}
files = os.listdir(TRAIN_DATA_DIR)
for fname in files:
    name, ext = os.path.splitext(fname)
    if name in testset:
        continue
        
    if ext == ".npz":
        if name + '.jpg' in files:
            img = Image.load(os.path.join(TRAIN_DATA_DIR, name) + '.jpg')
        elif name + '.png' in files:
            img = Image.load(os.path.join(TRAIN_DATA_DIR, name) + '.png')
        
        npzfname = os.path.join(TRAIN_DATA_DIR, fname)
        npzdata = np.load(npzfname)
        for c in COLORS:
            if npzdata[c].size > 0:
                mat = npzdata[c]
                mat = mat.reshape(-1, 3).astype(np.uint8)
                data[c].append(mat)
                
for c in COLORS:
    data[c] = np.vstack(data[c])

print('---- done ------')

---- done ------


In [4]:
N_DATA_PER_CLASS = 200000
APPEND_YCRCB = False
APPEND_BIAS = False
ONLY_YCRCB = False

labelmp = {
    'COLOR_STOP_SIGN_RED': 0,
    'COLOR_OTHER_RED': 1,
    'COLOR_ORANGE': 1,
    'COLOR_BROWN': 1,
    'COLOR_BLUE': 2,
    'COLOR_OTHER': 2
}
X, y = [], []
for ci, c in enumerate(COLORS):
    print(c, data[c].shape)
    
    rndidx = np.random.choice(data[c].shape[0], N_DATA_PER_CLASS, replace=False)
    x = data[c][rndidx, :]
        
    X.append(x)
    y.append(np.ones((N_DATA_PER_CLASS, 1)) * labelmp[c])
    
X = np.vstack(X).astype(np.float64)
y = np.vstack(y).astype(np.int32).reshape(-1)
print('-----------done------------')

COLOR_STOP_SIGN_RED (1582972, 3)
COLOR_OTHER_RED (2233843, 3)
COLOR_BROWN (10679301, 3)
COLOR_ORANGE (266480, 3)
COLOR_BLUE (45441366, 3)
COLOR_OTHER (181054804, 3)
-----------done------------


In [9]:
from sklearn.utils import shuffle

X, y = shuffle(X, y, random_state=1)
clf = classifier.GaussianNaiveBayes()
clf.fit(X, y)
clf.save('../model/gnb_test.pic')